In [1]:
import numpy as np
import os
import csv
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score, KFold
from sklearn.svm import SVC
import pandas as pd
from patsy import dmatrices
from sklearn import datasets, svm
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn import metrics

In [2]:
df = pd.read_csv("HW5_jobs_data/train_data.csv")

df_train = df.drop(['JobID','Employer', 'class'], axis=1)
df_class = df["class"]

# Using SVM

In [3]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(df_train, df_class, test_size=0.4, random_state=0)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
clf = svm.SVC(kernel='linear', C=0.03125, gamma=0.5,degree=10).fit(X_train_transformed, y_train)
X_test_transformed = scaler.transform(X_test)
clf.score(X_test_transformed, y_test)

0.89344262295081966

In [4]:
predicted = cross_validation.cross_val_predict(clf, df_train, df_class, cv=31)
metrics.accuracy_score(df_class, predicted)

0.85550082101806235

### Using SVM predictions were about 17% DS jobs 

In [5]:
df = pd.read_csv("HW5_jobs_data/test_data.csv")
df_test = df.drop(['JobID','Employer'], axis=1)
df_test_transformed = scaler.transform(df_test)
pred = clf.predict(df_test_transformed)

# Using SVM Pipeline

In [6]:
n_samples = df_train.shape[0]
cv = cross_validation.ShuffleSplit(n_samples, n_iter=3, test_size=0.1, random_state=0)
pip_clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(kernel='linear', C=0.03125, gamma=0.5,degree=10))
scores = cross_validation.cross_val_score(pip_clf, df_train, df_class, cv=cv)
print scores.mean()

0.912568306011


In [7]:
predicted = cross_validation.cross_val_predict(pip_clf, df_train, df_class, cv=31)
metrics.accuracy_score(df_class, predicted) 
#print predicted

0.909688013136289

In [8]:
df = pd.read_csv("HW5_jobs_data/test_data.csv")
df_test = df.drop(['JobID','Employer'], axis=1)

In [9]:
pip_clf.fit(df_train, df_class)
pred_pip = pd.DataFrame(pip_clf.predict(df_test))

### Using SVM Pipeline the predictions had about 20% DS jobs and 80% non DS jobs

In [10]:
pred_pip.to_csv('test_predictions.csv',index=False, column=False)

In [11]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import numpy as np


cols_names = df_train.columns
cols_names = cols_names.map(lambda x: x.replace(' ', '_') if isinstance(x, (str, unicode)) else x)
df_train.columns = cols_names
#print cols

best_features = SelectKBest(chi2, k=10)            
XyTrain = best_features.fit_transform(df_train,df_class)

feature_names = df_train.columns[best_features.get_support()]

cols = feature_names
cols = cols.map(lambda x: x.replace(' ', '_') if isinstance(x, (str, unicode)) else x)
feature_names = cols
feature_names

array(['big_data', 'data', 'data_science', 'hadoop_tag', 'scala_tag',
       'spark_tag', 'bigdata_tag', 'machinelearning_tag', 'oop_tag',
       'warehouse_tag'], dtype=object)

In [12]:
top_10_features = pd.DataFrame(feature_names)
top_10_features
top_10_features.to_csv('top_10_features.csv', header=False, index=False)